In [1]:
#disease_duration   24.01.30
#
#Perform Kruskal–Wallis test to see the difference between ACPA– and ACPA+ RA.
#EAC (Early arthritis cohort = disease duration = 0 )
#ACPA+ RA = 1
#ACPA– RA = 2

#This analysis was done after recieving internal review from the Drs.
#And will be inserted into Table 1

In [13]:
data_file = "../../analysis_addressing_feedback/disease_duration/data/disease_duration.csv"
data_df = read.csv(data_file, sep=",")
data_df$disease_duration[data_df$disease_duration == "EAC"] <- 0
data_df$disease_duration <- as.integer(data_df$disease_duration)

In [14]:
kruskal.test(disease_duration ~ ACPA, data = data_df)


	Kruskal-Wallis rank sum test

data:  disease_duration by ACPA
Kruskal-Wallis chi-squared = 0.068364, df = 1, p-value = 0.7937
